In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import joblib
import pickle

import os

from datetime  import datetime

import collections



In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

# Critical Parameter

In [21]:
strategyName='short'

algoName= 'XGB'#'GBM'#

st='01-2007'
ed='03-2021'
print(f"My Train {st} - {ed}")

st_test='03-2021'
ed_test='12-2021'
print(f'My Test {st_test} - {ed_test}')

predictionColName="Signal"

isFullFeatures=True


prob_base_score=0.65

My Train 01-2007 - 03-2021
My Test 03-2021 - 12-2021


In [22]:
# # trail 2007-2q20 Ver2 July2020
# if strategyName=='long':
#  nTree=80
#  learnRate=0.01
# elif strategyName=='short':
#  nTree=160
#  learnRate=0.05

# mxDepth=8

# print(strategyName,' - ' 'parameter model : ',nTree,'-',learnRate,'-',mxDepth)

In [23]:
# trail 2007-1q21
if strategyName=='long':
 nTree=100
 learnRate=0.05
elif strategyName=='short':
 nTree=120
 learnRate=0.005

mxDepth=8

print(strategyName,' - ' 'parameter model : ',nTree,'-',learnRate,'-',mxDepth)

short  - parameter model :  120 - 0.005 - 8


In [24]:
folder_model=f'models-{strategyName}'
print(f'folder_model={folder_model}')


if isFullFeatures==False:
  featureTitle=f'selectFeat'
else:
  featureTitle='allFeat'


print(f'select feature={featureTitle}')

folder_model=models-short
select feature=allFeat


In [25]:
model = XGBClassifier(n_estimators=nTree,max_depth=mxDepth,learning_rate=learnRate,
                      importance_type='gain',base_score=prob_base_score)

print(model)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.005, max_delta_step=None, max_depth=8,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=120, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)


In [26]:

#feat_path=r'D:\DataSets\FeatSelection\xgb_s50f15m_featSelect_mini.xlsx'   
feat_path=r'D:\DataSets\FeatSelection\xgb_s50f15m_featSelect.xlsx'  
if strategyName=='long':
 sheet_name_feat="long_xgb"
 pathname=r'D:\DataSets\ML-Long_S50M15_07To1Q21.csv'
else :
 sheet_name_feat="short_xgb"
 pathname=r'D:\DataSets\ML-Short_S50M15_07To1Q21.csv'

print(pathname)
print(feat_path)

D:\DataSets\ML-Short_S50M15_07To1Q21.csv
D:\DataSets\FeatSelection\xgb_s50f15m_featSelect.xlsx


In [27]:
filepath= os.path.abspath(pathname)
dataset =pd.read_csv(pathname,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75006 entries, 2007-01-03 09:45:00 to 2021-03-31 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    75006 non-null  float64
 1   high                    75006 non-null  float64
 2   low                     75006 non-null  float64
 3   close                   75006 non-null  float64
 4   indy_ma-220             75006 non-null  float64
 5   indy_ma-550             75006 non-null  float64
 6   indy_hh-220             75006 non-null  float64
 7   indy_ll-220             75006 non-null  float64
 8   indy_mid-220            75006 non-null  float64
 9   indy_hh2-550            75006 non-null  float64
 10  indy_ll2-550            75006 non-null  float64
 11  indy_mid2-550           75006 non-null  float64
 12  indy_macd110-440        75006 non-null  float64
 13  indy_signal110-440-110  75006 non-null  float64
 14  ind

In [28]:
# x[-1] is lable always
label=dataset.columns[len(dataset.columns)-1]
print('label:',label)

label: ShortSignal


In [29]:

if  isFullFeatures==True:
    print("FullFeatures")
    select_feat=np.array(dataset.columns )
else:
    print("Select Features")
    feat_path=os.path.abspath(feat_path)
    feat_df=pd.read_excel(feat_path,header=None,engine='openpyxl',sheet_name=sheet_name_feat)
    #print('feature selection : ',feat_df)
    select_feat=feat_df.loc[:,0].tolist()
    select_feat.extend(['open','high','low','close'])
    select_feat.append(label)
    select_feat=np.array(select_feat )
    

print(type(select_feat))
print(select_feat)


FullFeatures
<class 'numpy.ndarray'>
['open' 'high' 'low' 'close' 'indy_ma-220' 'indy_ma-550' 'indy_hh-220'
 'indy_ll-220' 'indy_mid-220' 'indy_hh2-550' 'indy_ll2-550'
 'indy_mid2-550' 'indy_macd110-440' 'indy_signal110-440-110'
 'indy_hist_macd110-440' 'indy_rsi25-ma20' 'indy_6ATRTrail_DC-110'
 'cate_3trend-220_ma110' 'cate_2trend-550_ma110' 'cate_rannkHL550-ma66'
 'cate_CombineTrend' 'ShortSignal']


In [30]:
def CreateData(dfx):
    
 labelCol=dfx.columns[len(select_feat)-1]
 print(labelCol)
 
 X_df_temp =dfx.drop(columns=['open','high','low','close',labelCol])
 Y_df_temp = dfx[labelCol]

 Xtemp=X_df_temp.to_numpy()
 ytemp=Y_df_temp.to_numpy()

 return X_df_temp,Y_df_temp, Xtemp,ytemp


# Filter train period

In [31]:
df_train=dataset.loc[st:ed,select_feat]

X_df,Y_df,X,y=CreateData( df_train)

print(X_df.info())

ShortSignal
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75006 entries, 2007-01-03 09:45:00 to 2021-03-31 16:45:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-220             75006 non-null  float64
 1   indy_ma-550             75006 non-null  float64
 2   indy_hh-220             75006 non-null  float64
 3   indy_ll-220             75006 non-null  float64
 4   indy_mid-220            75006 non-null  float64
 5   indy_hh2-550            75006 non-null  float64
 6   indy_ll2-550            75006 non-null  float64
 7   indy_mid2-550           75006 non-null  float64
 8   indy_macd110-440        75006 non-null  float64
 9   indy_signal110-440-110  75006 non-null  float64
 10  indy_hist_macd110-440   75006 non-null  float64
 11  indy_rsi25-ma20         75006 non-null  float64
 12  indy_6ATRTrail_DC-110   75006 non-null  float64
 13  cate_3trend-220_ma110   75006 non-null  int6

In [32]:
X_df.tail(5)

,indy_ma-220,indy_ma-550,indy_hh-220,indy_ll-220,indy_mid-220,indy_hh2-550,indy_ll2-550,indy_mid2-550,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-220_ma110,cate_2trend-550_ma110,cate_rannkHL550-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2021-03-31 15:45:00,1.0,0.99,1.01,0.98,0.001,1.02,0.95,0.001,2.39,1.18,1.20,47.74,1.0,2,1,3,4
2021-03-31 16:00:00,1.0,0.99,1.01,0.98,0.001,1.02,0.95,0.001,2.45,1.16,1.29,47.00,1.0,2,1,3,4
2021-03-31 16:15:00,1.0,0.99,1.01,0.98,0.001,1.02,0.95,0.001,2.53,1.14,1.39,46.28,1.0,2,1,3,4
2021-03-31 16:30:00,1.0,0.99,1.01,0.98,0.001,1.02,0.95,0.001,2.58,1.12,1.46,46.08,1.0,2,1,3,4
2021-03-31 16:45:00,1.0,0.99,1.01,0.98,0.001,1.02,0.95,0.001,2.61,1.11,1.51,45.92,1.0,2,1,3,4


In [33]:
print(Y_df.tail(5))

datetime
2021-03-31 15:45:00    0
2021-03-31 16:00:00    0
2021-03-31 16:15:00    0
2021-03-31 16:30:00    0
2021-03-31 16:45:00    0
Name: ShortSignal, dtype: int64


In [34]:
print(X[:3])
print(y[:3])

[[ 1.050e+00  1.100e+00  1.140e+00  9.100e-01  2.200e-03  1.150e+00
   9.100e-01  2.200e-03 -2.250e+01 -2.720e+01  4.700e+00  3.911e+01
   1.030e+00  1.000e+00  0.000e+00  3.000e+00  2.000e+00]
 [ 1.040e+00  1.090e+00  1.120e+00  8.900e-01  2.200e-03  1.140e+00
   8.900e-01  2.200e-03 -2.250e+01 -2.719e+01  4.680e+00  3.905e+01
   1.020e+00  1.000e+00  0.000e+00  3.000e+00  2.000e+00]
 [ 1.050e+00  1.090e+00  1.130e+00  9.000e-01  2.200e-03  1.150e+00
   9.000e-01  2.200e-03 -2.253e+01 -2.717e+01  4.640e+00  3.891e+01
   1.030e+00  1.000e+00  0.000e+00  3.000e+00  2.000e+00]]
[0 0 0]


In [35]:
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.005, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=120, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# Filter Test Period

In [107]:
df_test=dataset.loc[st_test:ed_test,select_feat]
#df_train.info()
XTest_df,YTest_df,X_test,y_test=CreateData( df_test)

print(XTest_df.info())
print("====================================================")

print(YTest_df.tail(5))
XTest_df.tail(5)


LongSignal
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 507 entries, 2021-03-01 09:45:00 to 2021-03-31 16:45:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             507 non-null    float64
 1   indy_ma-1100            507 non-null    float64
 2   indy_hh-550             507 non-null    float64
 3   indy_ll-550             507 non-null    float64
 4   indy_mid-550            507 non-null    float64
 5   indy_hh2-1100           507 non-null    float64
 6   indy_ll2-1100           507 non-null    float64
 7   indy_mid2-1100          507 non-null    float64
 8   indy_macd110-440        507 non-null    float64
 9   indy_signal110-440-110  507 non-null    float64
 10  indy_hist_macd110-440   507 non-null    float64
 11  indy_rsi25-ma20         507 non-null    float64
 12  indy_6ATRTrail_DC-110   507 non-null    float64
 13  cate_3trend-550_ma110   507 non-null    int64  

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2021-03-31 15:45:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.39,1.18,1.20,47.74,1.0,3,0,3,4
2021-03-31 16:00:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.45,1.16,1.29,47.00,1.0,3,0,3,4
2021-03-31 16:15:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.53,1.14,1.39,46.28,1.0,3,0,3,4
2021-03-31 16:30:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.58,1.12,1.46,46.08,1.0,3,0,3,4
2021-03-31 16:45:00,0.99,0.98,1.02,0.95,0.001,1.02,0.94,0.001,2.61,1.11,1.51,45.92,1.0,3,0,3,4


In [108]:
df_ohlc=dataset.loc[st_test:ed_test,['open','high','low','close']]
df_ohlc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 507 entries, 2021-03-01 09:45:00 to 2021-03-31 16:45:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    507 non-null    float64
 1   high    507 non-null    float64
 2   low     507 non-null    float64
 3   close   507 non-null    float64
dtypes: float64(4)
memory usage: 19.8 KB


In [109]:
# make predictions for test data
yPredited = model.predict(X_test)
predictions = [round(value) for value in yPredited]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

from sklearn import metrics
pd.DataFrame(metrics.confusion_matrix(y_test, predictions),columns=['Acutal_SideWay','Actual_Long'],index=['Pred_SideWay','Pred_Long'])

Accuracy: 70.81%


,Acutal_SideWay,Actual_Long
Pred_SideWay,108,110
Pred_Long,38,251


In [110]:
prediction_df=pd.DataFrame(data= {predictionColName: predictions },index=df_ohlc.index)

df_result=pd.concat([df_ohlc,prediction_df],axis=1)

df_result.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 507 entries, 2021-03-01 09:45:00 to 2021-03-31 16:45:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    507 non-null    float64
 1   high    507 non-null    float64
 2   low     507 non-null    float64
 3   close   507 non-null    float64
 4   Signal  507 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 23.8 KB


In [111]:
df_result.head()

,open,high,low,close,Signal
datetime,,,,,
2021-03-01 09:45:00,918.9,922.5,917.2,919.9,0
2021-03-01 10:00:00,919.8,921.5,919.0,921.0,0
2021-03-01 10:15:00,921.1,922.2,920.7,921.7,0
2021-03-01 10:30:00,921.6,924.9,921.4,924.2,0
2021-03-01 10:45:00,924.1,924.7,923.3,923.5,0


In [112]:
yPredited_prob = model.predict_proba(X_test)
yPredited_prob=yPredited_prob*100
yPredited_prob[:10]

array([[99.085976  ,  0.9140245 ],
       [99.085976  ,  0.9140245 ],
       [99.07329   ,  0.9267077 ],
       [99.08827   ,  0.9117324 ],
       [99.234024  ,  0.7659797 ],
       [99.56877   ,  0.43122736],
       [99.52554   ,  0.47445217],
       [99.629715  ,  0.37029034],
       [99.59257   ,  0.40743208],
       [99.59257   ,  0.40743208]], dtype=float32)

In [113]:
df_predictResult=pd.DataFrame({'y_Actual':y_test,'y_Predicted':yPredited},index=df_ohlc.index)
df_predictResult.tail()

,y_Actual,y_Predicted
datetime,,
2021-03-31 15:45:00,0,1
2021-03-31 16:00:00,0,1
2021-03-31 16:15:00,0,1
2021-03-31 16:30:00,0,1
2021-03-31 16:45:00,0,1


In [114]:
df_predictProb=pd.DataFrame(data=yPredited_prob,columns=['Prob0','Prob_1'],index=df_ohlc.index)
df_predictProb.round(3).tail()


,Prob0,Prob_1
datetime,,
2021-03-31 15:45:00,18.000999,81.999001
2021-03-31 16:00:00,18.000999,81.999001
2021-03-31 16:15:00,17.878000,82.122002
2021-03-31 16:30:00,17.694000,82.306000
2021-03-31 16:45:00,17.694000,82.306000


In [115]:
df_AllResult=pd.concat([df_ohlc, df_predictResult,df_predictProb.round(3)],axis=1)

print(df_AllResult.info())
df_AllResult.sample(10)



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 507 entries, 2021-03-01 09:45:00 to 2021-03-31 16:45:00
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         507 non-null    float64
 1   high         507 non-null    float64
 2   low          507 non-null    float64
 3   close        507 non-null    float64
 4   y_Actual     507 non-null    int64  
 5   y_Predicted  507 non-null    int64  
 6   Prob0        507 non-null    float32
 7   Prob_1       507 non-null    float32
dtypes: float32(2), float64(4), int64(2)
memory usage: 31.7 KB
None


,open,high,low,close,y_Actual,y_Predicted,Prob0,Prob_1
datetime,,,,,,,,
2021-03-10 14:45:00,969.1,971.0,969.0,970.5,1,1,7.065000,92.934998
2021-03-19 16:15:00,957.5,958.3,956.7,957.2,1,1,46.363998,53.636002
2021-03-23 12:15:00,958.0,958.0,957.0,957.8,0,0,71.426003,28.573999
2021-03-25 15:15:00,966.7,968.0,965.3,965.5,0,1,12.804000,87.195999
2021-03-09 11:30:00,954.9,955.7,953.7,954.2,1,1,46.634998,53.365002
2021-03-02 11:45:00,929.6,931.7,929.6,931.2,0,0,97.750999,2.249000
2021-03-11 10:30:00,978.7,978.7,973.0,976.4,1,1,3.716000,96.283997
2021-03-25 11:15:00,960.1,960.5,959.7,960.1,0,1,17.329000,82.670998
2021-03-03 15:45:00,955.2,957.3,955.0,955.4,1,1,25.532000,74.468002


In [90]:

df_AllResult.to_csv(f'{strategyName}-{algoName}_{featureTitle}_{st_test}-{ed_test}.csv')

In [91]:
from sklearn.metrics import classification_report
print(classification_report(y_test,yPredited))

              precision    recall  f1-score   support

           0       0.83      0.84      0.84       754
           1       0.80      0.78      0.79       595

    accuracy                           0.81      1349
   macro avg       0.81      0.81      0.81      1349
weighted avg       0.81      0.81      0.81      1349



# Build Final Model

In [36]:
modelAlgo=f'{algoName}-{nTree}x{mxDepth}x{str(learnRate).replace(".","")}'

modelAlgo

'XGB-120x8x0005'

In [37]:
buildtime=datetime.now().strftime('%d%m%y_%H%M')
# save model to file


 #model_file=f'{folder_model}/{strategyName}_S50M15_{modelAlgo}_{st}t{ed}_{featureTitle}_b{buildtime}.joblib.dat'
model_file=f'{folder_model}/{strategyName}_S50M15_{modelAlgo}_{st}t{ed}_{featureTitle}_b{buildtime}.pickle.dat'

pickle.dump(model, open(model_file, "wb"))
#joblib.dump(model, model_file)

print(f"Saved model to: {model_file}")

Saved model to: models-short/short_S50M15_XGB-120x8x0005_01-2007t03-2021_allFeat_b170421_2320.pickle.dat
